In [75]:
import sys
sys.path.append("modules/")

import multicellds
from matplotlib import pyplot as plt
import seaborn as sns


In [79]:
output_folder = "test/"

mcds = multicellds.MultiCellDS(output_folder=output_folder)

# PhysiCell/Boss integer codes and the corresponding string the description
phases_dict = mcds.phases_dict

# The following dict is used to group cell phases into three general classes:
# Alive, Apoptotic (programmed cell death), Necrotic (Non-programmed cell death)
phase_grouping = mcds.phase_grouping


# Just counting the number of files (each one corresponding to a time snapshots)
num_of_files = mcds.cells_file_count()

# Initializing a Pandas Databrafe to store the data
columns = ["Time", "Alive", "Apoptotic", "Necrotic"]
data = np.zeros((num_of_files, 4), dtype=int)
df_time_course = pd.DataFrame(columns=columns, data=data)

print("Reading cell_output files from %i input files from %s" % (num_of_files, output_folder))
# Iterating over all cell_output files

phase_col = "current_phase"

for i, (t, df) in enumerate(mcds.cells_as_frames_iterator()):
    print("\tProcessing time step: %.0f" % t)

    time = int(np.ceil(t))

    # Rename the phases integer codes using the phases_dict as the mapping
    s = df[phase_col]
    s.replace(to_replace=mcds.phases_dict, value=None, inplace=True)

    # Count the number of cells in each phase
    counts = s.value_counts()

    df_time_course.iat[i, 0] = time
    # group the previous phases count into the three general classes:
    # Alive, Apoptotic, Necrotic
    for k, v in counts.to_dict().items():
        if k not in mcds.phase_grouping:
            continue
        df_time_course.loc[i, mcds.phase_grouping[k]] += v

# Set time column as the dataframe index
df_time_course.set_index("Time", inplace=True)

Reading cell_output files from 5 input files from test/
	Processing time step: 0
	Processing time step: 60
	Processing time step: 120
	Processing time step: 180
	Processing time step: 240


In [ ]:
print("Creating figure")
# Create a figure
fig, ax = plt.subplots(1, 1, figsize=(6,4), dpi=150)

# plot Alive vs Time
ax.plot(df_time_course.index, df_time_course.Alive, "-", c="g", label="alive")

# plot Necrotic vs Time
ax.plot(df_time_course.index, df_time_course.Necrotic, "-", c="k", label="necrotic")

# plot Apoptotic vs Time
ax.plot(df_time_course.index, df_time_course.Apoptotic, "-", c="r", label="apoptotic")

# setting axes labels
ax.set_xlabel("time (min)")
ax.set_ylabel("Nº of cells")

# Showing legend
ax.legend()
ax.grid(True)

sns.despine()
fig.tight_layout()